In [1]:
import pandas as pd
import pipeline as p
import datetime as dt

%load_ext autoreload
%autoreload 2

### Load 'merged' df (game/season stats + boxscore stats)

In [2]:
df = p.open_pkl('Data/merged_df.pkl')

## Clean further

### Convert team abbrevs to those used in Elo data table

In [3]:
elo_stats_team_dict = p.open_pkl('TeamDicts/elo_stats_team_dict.pkl')
corr_dict = p.open_pkl('TeamDicts/moved_teams_corr_dict.pkl')
full_name_dict = p.open_pkl('TeamDicts/full_name_team_dict.pkl')

# change the abbreviations of teams that moved in 2018
full_name_dict['San Diego Chargers'] = 'LAC'
full_name_dict['St. Louis Rams'] = 'LAR'

# create reverse-lookup of elo_stats
stats_elo_team_dict = {v:k for k,v in elo_stats_team_dict.items()}
stats_elo_team_dict['SDG'] = 'LAC'
stats_elo_team_dict['STL'] = 'LAR'

In [4]:
# stats_df['Team'] = stats_df['Team'].apply(lambda x: corr_dict[x] if x in corr_dict.keys() else x)
df['Opp'] = df['Opp'].apply(lambda x: full_name_dict[x])
df['Team'] = df['Team'].apply(lambda x: stats_elo_team_dict[x] if x in stats_elo_team_dict else x)
df['Opp'] = df['Opp'].apply(lambda x: stats_elo_team_dict[x] if x in stats_elo_team_dict else x)

### Clean up datatypes

In [5]:
# sort by date. Reset Index. Change name of 'Year' to 'Season' to avoid confusion
df.sort_values('Date', inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'Year':'Season'}, inplace=True)

In [6]:
# Convert to integers:

# Location: 1 = Home, 0 = Away
df['Location'] = df['Location'].apply(lambda x: 1 if x == '@' else 0)

# OT: 1 = OT occured. 0 = no OT. 
df['OT'] = df['OT'].apply(lambda x: 1 if x == 'OT' else 0)

# Result: 1 = Win, 0 = Loss
df['Result'] = df['Result'].apply(lambda x: 1 if x == 'W' else (0 if x == 'L' else 0.5))

In [7]:
# Time of Possession: convert to float of minutes
def convert_timeposs(row):
    return row.minute + (row.second/60)

df['TimePoss'] = pd.to_datetime(df['Time of Possession'], format='%M:%S').dt.time
# df.drop(columns={'Time of Possession'}, inplace=True)

df['TimePossMins'] = df['TimePoss'].apply(lambda x: convert_timeposs(x))

df.drop(columns=['TimePoss', 'Time of Possession'], inplace=True)

### Extract / Create features

In [8]:
df.dropna(subset=['Record'], inplace=True)

df['Wins'] = df['Record'].apply(lambda row: p.row_split(row, 0))
df['Losses'] = df['Record'].apply(lambda row: p.row_split(row, 1))
df['Games'] = df['Wins'] + df['Losses']
df['Win%'] = df['Wins']/df['Games'] * 100

In [9]:
# shift wins, losses, games, win%
def correct_winloss(row, col):
    if row['Week'] == 1:
        return 0
    else:
        return row[col]

df['Wins'] = df.groupby(['Team'])['Wins'].shift()
df['Losses'] = df.groupby(['Team'])['Losses'].shift()
df['Games'] = df.groupby(['Team'])['Games'].shift()
df['Win%'] = df.groupby(['Team'])['Win%'].shift()

df['Wins'].fillna(0, inplace=True)
df['Losses'].fillna(0, inplace=True)
df['Games'].fillna(0, inplace=True)
df['Win%'].fillna(0, inplace=True)

df['Wins'] = df.apply(lambda row: correct_winloss(row, 'Wins'), axis=1)
df['Losses'] = df.apply(lambda row: correct_winloss(row, 'Losses'), axis=1)
df['Games'] = df.apply(lambda row: correct_winloss(row, 'Games'), axis=1)
df['Win%'] = df.apply(lambda row: correct_winloss(row, 'Win%'), axis=1)

In [10]:
# Passer stats
df['PassComp%'] = df['PassCmp'] / df['PassAtt'] * 100

def passer_rating(row):
    a = (row['PassCmp']/row['PassAtt'] - 0.3) * 5
    b = (row['PassY']/row['PassAtt'] - 3) * 0.25
    c = row['PassTDs']/row['PassAtt'] * 20
    d = 2.375 - (row['INT']/row['PassAtt'] * 25)
    return (a+b+c+d)/6 * 100

df['PassRating'] = df.apply(lambda row: passer_rating(row), axis=1)

In [11]:
df.drop(columns='index', inplace = True)
df.rename(columns={'neutral':'Neutral'}, inplace=True)

In [13]:
df.loc[0]

1stD                             18
3rdDAtt                          15
3rdDConv                          6
4thDAtt                           1
Date            2007-09-06 00:00:00
Day                             Thu
DefTO                             1
Fumbles                           2
INT                               2
Location                          1
OT                                0
Opp                             IND
PassAtt                          41
PassCmp                          28
PassTDs                           0
PassY                           187
PenY                             20
Penalies                          4
PtsOpp                           41
PtsTm                            10
Record                          0-1
Result                            0
RushAtt                          26
RushTDs                           0
RushY                           106
SacksO                            1
TO                                3
Team                        

Drop Record?

In [14]:
p.pkl_this('Data/merged_df_cleaned.pkl', df)

In [56]:
df.head()

,1stD,3rdDAtt,3rdDConv,4thDAtt,Date,Day,DefTO,Fumbles,INT,Location,...,TotY,Week,Season,TimePossMins,Wins,Losses,Games,Win%,PassComp%,PassRating
0,18.0,15.0,6.0,1.0,2007-09-06,Thu,1.0,2.0,2.0,@,...,293.0,1,2007,31.750000,0.0,0.0,0.0,0.0,68.292683,57.672764
1,20.0,11.0,4.0,0.0,2007-09-06,Thu,3.0,1.0,0.0,NaN,...,452.0,1,2007,28.250000,0.0,0.0,0.0,0.0,60.000000,125.416667
2,22.0,14.0,7.0,2.0,2007-09-09,Sun,2.0,1.0,1.0,@,...,438.0,1,2007,32.050000,0.0,0.0,0.0,0.0,65.909091,107.575758
3,21.0,13.0,6.0,0.0,2007-09-09,Sun,3.0,0.0,2.0,@,...,392.0,1,2007,26.900000,0.0,0.0,0.0,0.0,75.000000,102.083333
4,14.0,11.0,3.0,1.0,2007-09-09,Sun,2.0,3.0,2.0,@,...,219.0,1,2007,27.166667,0.0,0.0,0.0,0.0,66.666667,50.946970


### Compute trailing season average stats

In [21]:
stats_df = df.copy()

In [19]:
stats_df.columns

Index(['Date', 'Day', 'Location', 'OT', 'Opp', 'Record', 'Result', 'Team',
       'Time', 'Week', 'Season', 'TimePossMins', 'Wins', 'Losses', 'Games',
       'Win%', '1stD_SA', 'PassY_SA', 'PtsOpp_SA', 'PtsTm_SA', 'RushY_SA',
       'TO_SA', 'DefTO_SA', 'TotY_SA', 'RushAtt_SA', 'RushTDs_SA',
       'PassCmp_SA', 'PassAtt_SA', 'PassTDs_SA', 'INT_SA', 'SacksO_SA',
       'Fumbles_SA', 'Penalies_SA', 'PenY_SA', '3rdDConv_SA', '3rdDAtt_SA',
       '4thDAtt_SA', 'PassComp%_SA', 'PassRating_SA'],
      dtype='object')

In [22]:
sa_cols_drop_true = ['1stD', 'PassY', 'PtsOpp',
       'PtsTm', 'RushY', 'TO', 'DefTO', 
       'TotY', 'RushAtt', 'RushTDs', 'PassCmp', 'PassAtt',
       'PassTDs', 'INT', 'SacksO', 'Fumbles', 'Penalies', 'PenY', '3rdDConv',
       '3rdDAtt', '4thDAtt',  
       'PassComp%',
       'PassRating', 'TimePossMins']


for col in sa_cols_drop_true:
    stats_df = p.calc_season_avg(col, stats_df, drop=True) 

In [23]:
stats_df.head()

,Date,Day,Location,OT,Opp,Record,Result,Team,Time,Week,...,SacksO_SA,Fumbles_SA,Penalies_SA,PenY_SA,3rdDConv_SA,3rdDAtt_SA,4thDAtt_SA,PassComp%_SA,PassRating_SA,TimePossMins_SA
0,2007-09-06,Thu,1,0,IND,0-1,0.0,NO,20:39:00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-09-06,Thu,0,0,NO,1-0,1.0,IND,20:39:00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-09-09,Sun,1,0,DAL,0-1,0.0,NYG,20:25:00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-09-09,Sun,1,0,OAK,1-0,1.0,DET,16:15:00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-09-09,Sun,1,0,HOU,0-1,0.0,KC,13:05:00,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
p.pkl_this('Data/stats_df.pkl', stats_df)

Week 1 SA are blank. Pull in previous season's stats? Or revert them to an average?

### Merge with Elo ratings

In [70]:
elo_df = p.open_pkl('Data/elo_df.pkl')

In [71]:
elo_df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2
0,2007-09-06,2007,0,NaN,IND,NO,1653.923,1515.973,0.762833,0.237167,1668.974,1500.922,41.0,10.0
1,2007-09-09,2007,0,NaN,DAL,NYG,1490.892,1494.060,0.588056,0.411944,1510.108,1474.844,45.0,35.0
2,2007-09-09,2007,0,NaN,LAR,CAR,1478.632,1511.942,0.545480,0.454520,1448.656,1541.917,13.0,27.0
3,2007-09-09,2007,0,NaN,LAC,CHI,1646.204,1574.744,0.686870,0.313130,1660.857,1560.090,14.0,3.0
4,2007-09-09,2007,0,NaN,WSH,MIA,1448.841,1489.941,0.534341,0.465659,1461.613,1477.169,16.0,13.0


Compute Deltas (easier now that pre and post are provided)

In [75]:
elo_df['elo1_delta'] = elo_df['elo1_post'] - elo_df['elo1_pre']
elo_df['elo2_delta'] = elo_df['elo2_post'] - elo_df['elo2_pre']
elo_df['point_diff'] = elo_df['score1'] - elo_df['score2']

In [81]:
elo_df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2,elo1_delta,elo2_delta,point_diff
0,2007-09-06,2007,0,NaN,IND,NO,1653.923,1515.973,0.762833,0.237167,1668.974,1500.922,41.0,10.0,15.051,-15.051,31.0
1,2007-09-09,2007,0,NaN,DAL,NYG,1490.892,1494.060,0.588056,0.411944,1510.108,1474.844,45.0,35.0,19.216,-19.216,10.0
2,2007-09-09,2007,0,NaN,LAR,CAR,1478.632,1511.942,0.545480,0.454520,1448.656,1541.917,13.0,27.0,-29.976,29.975,-14.0
3,2007-09-09,2007,0,NaN,LAC,CHI,1646.204,1574.744,0.686870,0.313130,1660.857,1560.090,14.0,3.0,14.653,-14.654,11.0
4,2007-09-09,2007,0,NaN,WSH,MIA,1448.841,1489.941,0.534341,0.465659,1461.613,1477.169,16.0,13.0,12.772,-12.772,3.0


In [82]:
df2 = pd.merge(elo_df, stats_df, right_on=['Team','Date'], left_on=['team1','date'])
df2 = pd.merge(df2, stats_df, right_on=['Team','Date'], left_on=['team2','date'], suffixes=('1','2'))

In [83]:
list(df2.columns)

['date',
 'season',
 'neutral',
 'playoff',
 'team1',
 'team2',
 'elo1_pre',
 'elo2_pre',
 'elo_prob1',
 'elo_prob2',
 'elo1_post',
 'elo2_post',
 'score1',
 'score2',
 'elo1_delta',
 'elo2_delta',
 'point_diff',
 'Date1',
 'Day1',
 'Location1',
 'OT1',
 'Opp1',
 'Record1',
 'Result1',
 'Team1',
 'Time1',
 'Week1',
 'Season1',
 'TimePossMins1',
 'Wins1',
 'Losses1',
 'Games1',
 'Win%1',
 '1stD_SA1',
 'PassY_SA1',
 'PtsOpp_SA1',
 'PtsTm_SA1',
 'RushY_SA1',
 'TO_SA1',
 'DefTO_SA1',
 'TotY_SA1',
 'RushAtt_SA1',
 'RushTDs_SA1',
 'PassCmp_SA1',
 'PassAtt_SA1',
 'PassTDs_SA1',
 'INT_SA1',
 'SacksO_SA1',
 'Fumbles_SA1',
 'Penalies_SA1',
 'PenY_SA1',
 '3rdDConv_SA1',
 '3rdDAtt_SA1',
 '4thDAtt_SA1',
 'PassComp%_SA1',
 'PassRating_SA1',
 'Date2',
 'Day2',
 'Location2',
 'OT2',
 'Opp2',
 'Record2',
 'Result2',
 'Team2',
 'Time2',
 'Week2',
 'Season2',
 'TimePossMins2',
 'Wins2',
 'Losses2',
 'Games2',
 'Win%2',
 '1stD_SA2',
 'PassY_SA2',
 'PtsOpp_SA2',
 'PtsTm_SA2',
 'RushY_SA2',
 'TO_SA2',
 'Def

In [84]:
cols_to_drop = [
 'Date1',
 'Day1',
 'Location1',
 'Opp1',
 'Record1',
 'Result1',
 'Team1',
 'Season1',
 'Wins1',
 'Losses1',
 'Games1',
 'Date2',
 'Day2',
 'Location2',
 'OT2',
 'Opp2',
 'Record2',
 'Result2',
 'Team2',
 'Time2',
 'Week2',
 'Season2',
 'Wins2',
 'Losses2',
 'Games2']

In [85]:
df2.rename(columns={'OT1':'OT','Week1':'week','Time1':'time'}, inplace=True)

In [86]:
df2.drop(columns=cols_to_drop, inplace=True)

In [360]:
df2.shape

(2929, 66)

In [87]:
list(df2.columns)

['date',
 'season',
 'neutral',
 'playoff',
 'team1',
 'team2',
 'elo1_pre',
 'elo2_pre',
 'elo_prob1',
 'elo_prob2',
 'elo1_post',
 'elo2_post',
 'score1',
 'score2',
 'elo1_delta',
 'elo2_delta',
 'point_diff',
 'OT',
 'time',
 'week',
 'TimePossMins1',
 'Win%1',
 '1stD_SA1',
 'PassY_SA1',
 'PtsOpp_SA1',
 'PtsTm_SA1',
 'RushY_SA1',
 'TO_SA1',
 'DefTO_SA1',
 'TotY_SA1',
 'RushAtt_SA1',
 'RushTDs_SA1',
 'PassCmp_SA1',
 'PassAtt_SA1',
 'PassTDs_SA1',
 'INT_SA1',
 'SacksO_SA1',
 'Fumbles_SA1',
 'Penalies_SA1',
 'PenY_SA1',
 '3rdDConv_SA1',
 '3rdDAtt_SA1',
 '4thDAtt_SA1',
 'PassComp%_SA1',
 'PassRating_SA1',
 'TimePossMins2',
 'Win%2',
 '1stD_SA2',
 'PassY_SA2',
 'PtsOpp_SA2',
 'PtsTm_SA2',
 'RushY_SA2',
 'TO_SA2',
 'DefTO_SA2',
 'TotY_SA2',
 'RushAtt_SA2',
 'RushTDs_SA2',
 'PassCmp_SA2',
 'PassAtt_SA2',
 'PassTDs_SA2',
 'INT_SA2',
 'SacksO_SA2',
 'Fumbles_SA2',
 'Penalies_SA2',
 'PenY_SA2',
 '3rdDConv_SA2',
 '3rdDAtt_SA2',
 '4thDAtt_SA2',
 'PassComp%_SA2',
 'PassRating_SA2']

In [89]:
p.pkl_this('Data/merged_by_game_df.pkl', df2)

In [88]:
df2.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,INT_SA2,SacksO_SA2,Fumbles_SA2,Penalies_SA2,PenY_SA2,3rdDConv_SA2,3rdDAtt_SA2,4thDAtt_SA2,PassComp%_SA2,PassRating_SA2
0,2007-09-06,2007,0,NaN,IND,NO,1653.923,1515.973,0.762833,0.237167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-09-09,2007,0,NaN,DAL,NYG,1490.892,1494.060,0.588056,0.411944,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-09-09,2007,0,NaN,LAR,CAR,1478.632,1511.942,0.545480,0.454520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-09-09,2007,0,NaN,LAC,CHI,1646.204,1574.744,0.686870,0.313130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-09-09,2007,0,NaN,WSH,MIA,1448.841,1489.941,0.534341,0.465659,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
